In [33]:
%%time
import sys
sys.path.append('./libs')
import util_modelo_referencial, util_bd, util_fasta, util_caracteristicas
import warnings
warnings.filterwarnings("ignore")

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.externals.joblib import dump, load
from sklearn.utils import shuffle
import random

import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import string
from tempfile import mkdtemp

class GeneradorFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, identificador=None):
        self.identificador = identificador
        self.random_id = identificador + "_fold_" + ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
        
    def fit(self, X, y=None):
        identificador = self.random_id
        codigos_lncRNA = {}
        codigos_PCT = {}
        for i in range(len(X)):
            if y[i] == 0:
                codigos_lncRNA[X[i][0]] = X[i][1]
            else:
                codigos_PCT[X[i][0]] = X[i][1]
        util_caracteristicas.generar_modelo_CPAT(identificador, codigos_lncRNA, codigos_PCT)
        return self
        
    def transform(self, X):
        identificador = self.random_id
        codigos = {codigo[0]:codigo[1] for codigo in X}
        dict_features = util_caracteristicas.generar_caracteristicas(identificador, codigos)

        return [list(x.values()) for x in dict_features.values()]

def crear_modelo_referencial(identificador, tuned_parameters, scores, n_jobs):
    codigos_lncRNA = util_fasta.leer_fasta("./data/" + identificador + ".lncRNA.fasta")
    codigos_PCT = util_fasta.leer_fasta("./data/" + identificador + ".PCT.fasta")
    
    codigos_lncRNA = {k: codigos_lncRNA[k] for k in random.sample(codigos_lncRNA.keys(),101)}
    codigos_PCT = {k: codigos_PCT[k] for k in random.sample(codigos_PCT.keys(),101)}
    
    #util_caracteristicas.generar_modelo_CPAT(identificador, codigos_lncRNA.keys(), codigos_PCT.keys())
    
    #dict_features_lncRNA = util_caracteristicas.generar_caracteristicas(identificador, codigos_lncRNA)
    #dict_features_PCT = util_caracteristicas.generar_caracteristicas(identificador, codigos_PCT)
    
    #features_lncRNA = [list(x.values()) for x in dict_features_lncRNA.values()]
    #features_PCT = [list(x.values()) for x in dict_features_PCT.values()]
    
    #X = features_lncRNA + features_PCT
    #y = ([1] * len(features_lncRNA)) + ([0] * len(features_PCT))
    #X_train, y_train = shuffle(X, y, random_state=0)
    
    X = list(codigos_lncRNA.items()) + list(codigos_PCT.items())
    y = ([1] * len(codigos_lncRNA)) + ([0] * len(codigos_PCT))
    X_train, y_train = shuffle(X, y, random_state=0)
    cachedir = mkdtemp()
    svm_pipeline = Pipeline(steps=[('features', GeneradorFeatures(identificador)), ('svc', SVC())], memory=cachedir)
    
    for score in scores:
        clf = GridSearchCV(svm_pipeline, tuned_parameters, cv=3, scoring=score, n_jobs=n_jobs, refit="accuracy")
        clf.fit(X_train, y_train)
        resultado = {
            "accuracy" : clf.cv_results_['mean_test_accuracy'][clf.best_index_],
            "precision" : clf.cv_results_['mean_test_precision'][clf.best_index_],
            "recall" : clf.cv_results_['mean_test_recall'][clf.best_index_]
        }
        print(resultado)

print("Iniciando proceso...")

query = "SELECT m.id_especie, m.especie FROM especies_seleccionadas s JOIN maestra_especies m ON s.especie = m.especie LIMIT 1"
especies = util_bd.resultados_query(query)

scores = [['accuracy','precision','recall']]
              
for especie in especies:
    id_especie = especie[0]
    identificador = "Especie{}".format(id_especie)
    tuned_parameters = [{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3], 'svc__C': [0.9, 1], 'features__identificador': [identificador]}]
    crear_modelo_referencial(identificador, tuned_parameters, scores, n_jobs=-1)

print("Proceso terminado...")

Iniciando proceso...
INI Especie2
CREANDO
INSTANCIANDO ID: Especie2NLB66
FIN CREANDO
ID AHORITA ES Especie2
INSTANCIANDO ID: Especie21LYYQ
INSTANCIANDO ID: Especie2HEI60
INSTANCIANDO ID: Especie275TMY
INSTANCIANDO ID: Especie2118XB
INSTANCIANDO ID: Especie2D59NB
INSTANCIANDO ID: Especie2P5ULX
INSTANCIANDO ID: Especie283JXY
INSTANCIANDO ID: Especie2LCKTZ
GENERANDO MODELO
ID: Especie2LCKTZ
GENERANDO FEATURES
ID: Especie2LCKTZ
{'accuracy': 0.8366336633663366, 'precision': 0.8545156935048344, 'recall': 0.8118811881188119}
Proceso terminado...
